In [73]:
!pip install pymongo
from pymongo import MongoClient
import pymongo

In [74]:
#before running the code, please read each code block's intructions carefully. 

#Once you open up MongoDB Compass, copy the connection string below to connect to Compass. 
#Download MongoDB Community Edition to host a local cluster: https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-windows/ 
#An easy tutorial for Mac users: https://medium.com/create-a-clocking-in-system-on-react/creating-a-local-mongodb-database-and-insert-a-document-c6a4a2102a22
#Once you have MongoDB, be sure you can establish a local connection to Compass using the connection string: "mongodb://localhost:27017/"

client = MongoClient("mongodb://localhost:27017/")

In [75]:
#Inserting corona-out-3's raw data. This is just to view the all of the raw data. 

In [95]:
#Define the database "my_final_database" and the collection "tweets".

db = client["my_final_database"]
collection_name = db["tweets"]

In [77]:
#The following code opens the corona-out-3 file and loads it into "parsed_data and writes it onto disk.
#Expect this to take a while. 

#The first part loads the data. 
#Note that due to the nature of the original file, each line must be split as shown below. 
#You should modify the code below with the directory to your "corona-out-3" file

import json

with open('/Users/mazinrafi/Downloads/corona-out-3', 'r') as f:
    parsed_data = []
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        parsed_data.append(obj)

        
#This second part writes the parsed file as a json file to disk. This file is called corona-out-2.json
#You may modify the file's extension to where you would like. This is saved to my download files. 
with open('/Users/mazinrafi/Downloads/corona-out-3.json', 'w') as f:
    json.dump(parsed_data, f)

In [96]:
#This will load the json file as "file_data" and uses the MongoDB "insert_many" command to insert the tweets. 
#After running this, the db "my_final_database" and "tweets" collection should be viewable on MongoDB Compass. 

#First we read the newly created json file from disk. As expected, this should take a while. 

import json
with open('/Users/mazinrafi/Downloads/corona-out-3.json') as f:
    file_data = json.load(f)
#The total number of tweets is 108596 if you run collection_name.insert_many(file_data). But is that actually true?
    
    
#The actual number of tweets with a unique ID is 108594, meaning 2 documents don't
#have a document id associated with it
We see this here: 
for item in file_data:
    # Insert or update the document in the collection using the "id" field as the unique identifier
    collection_name.replace_one({'id': item['id']}, item, upsert=True)

In [12]:
#!pip install scikit-learn
#!pip install nltk
#import nltk
#nltk.download('stopwords')

In [106]:
#Main setup for the App. This uses the TDM Creation code:

import json
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np

f= open('/Users/mazinrafi/Downloads/corona-out-3','r')

tweets = {}
texts = {}
users = {}
hashtags = {}

#aside: we might choose to see if verified=True too if you would like
tweet_keys = ['created_at', 'id_str', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count']
user_keys = ['name', 'screen_name', 'verified', 'followers_count', 'friends_count', 'created_at']

for line in f:
    if line == '\n':
        continue
    lx = json.loads(line)
    li = dict((k, lx[k]) for k in tweet_keys if k in lx)
    li['user'] = lx['user']['id']
    
    if not 'retweeted_status' in lx or len(lx['retweeted_status']) == 0:
        li['retweet'] = False
        if 'extended_tweet' in lx:    
            thisText = lx['extended_tweet']['full_text']
        else:
            thisText = lx['text']
        li['text'] = thisText
        texts[lx['id']] = thisText
        hashtag_set = list(set(part[1:] for part in thisText.split() if part.startswith('#')))
        li['hashtags'] = hashtag_set
        for hashtag in hashtag_set:
            if hashtag in hashtags.keys():
                hashtags[hashtag].append(lx['id'])
            else:
                hashtags[hashtag] = [lx['id']]
    else:
        li['retweet'] = True
        li['original_tweet'] = lx['retweeted_status']['id']
    tweets[lx['id']] = li
    if not lx['user']['id'] in users.keys():
        users[lx['user']['id']] = dict((k, lx['user'][k]) for k in user_keys if k in lx['user'])

f.close()

vectorizer = CountVectorizer(token_pattern=r"(?u)\b[a-z][a-z]+\b", stop_words=stopwords.words('english'), min_df = 500)

X = np.transpose(vectorizer.fit_transform(texts.values()))

key_list = [*texts.keys()]
word_list = [*vectorizer.get_feature_names_out()]

X

x1 = X.toarray()

x2 = dict((word_list[w], [key_list[t] for t in np.where(x1[w]>0)[0]]) for w in range(len(x1)))


with open("/Users/mazinrafi/Downloads/overall_tdm.json", "w") as outfile:
    json.dump(x2, outfile, indent=2)

with open("/Users/mazinrafi/Downloads/hashtag_tdm.json", "w") as outfile:
    json.dump(hashtags, outfile, indent=2)

# upload tweets, x2 to mongodb, users to mysql

# s1 = [sum(i>0) for i in x1]

# s1 = [np.log(i) for i in s1]

# from matplotlib import pyplot as plt

# fig, ax = plt.subplots(figsize =(10, 7))
# ax.hist(s1)


In [107]:
db = client['TwitterDB']
collection = db['tweets']

# Insert tweets into collection
for tweet_id, tweet_data in tweets.items():
    collection.insert_one(tweet_data)
    


In [108]:
len(tweets)

101894

In [38]:
#Upload the TDM json onto the server

client = MongoClient("mongodb://localhost:27017/")

# Create a new database
db = client["TwitterDB"]

# Create a new collection
collection = db["TDM"]

# Load the data from the JSON file
with open('/Users/mazinrafi/Downloads/overall_tdm.json') as f:
     = json.load(f)

# Insert the data into the collection
collection.insert_one(data)

In [59]:
#Upload the Hashtag json onto the server

client = MongoClient("mongodb://localhost:27017/")

# Create a new database
db = client["TwitterDB"]

# Create a new collection
collection = db["Hashtags"]

# Load the data from the JSON file
with open('/Users/mazinrafi/Downloads/hashtag_tdm.json') as f:
    data = json.load(f)

# Insert the data into the collection
collection.insert_one(data)

In [105]:
import pymongo
import json
from pymongo import MongoClient
import pandas as pd

# Perform the query to find the top 10 tweets based on favorite_count. Note that this 
client = MongoClient("mongodb://localhost:27017/")
db = client['TwitterDB']
collection = db['tweets']

top_tweets = db.tweets.find().sort("favorite_count", pymongo.DESCENDING).limit(10)

for tweet in top_tweets:
    if 'text' in tweet:
        print("Text:", tweet['text'])
        print("Favorite count:", tweet['favorite_count'])
        print("Retweet count:", tweet['retweet_count'])

Text: im making 17 a hr doing nothing 😭😭😭😭 i love this corona pay
Favorite count: 0
Retweet count: 0
Text: @VinceMcMahon @TripleH We hereby honor to announce that we have developed a brand new treatment method using UV rays / Laser Ray that might be used effectively for preventing all mankind from infectious virus.

#covid #covid19 #Covid_19 #corona #vascular #endovascular https://t.co/YkVlYtJiGL
Favorite count: 0
Retweet count: 0
Text: É isto, ou vou morrer sem ar ou com o corona https://t.co/O0Y7B3Koj4
Favorite count: 0
Retweet count: 0
Text: Schöne Runde mit dem Rennrad ✌️
#sport #corona https://t.co/Uglj9YJPI1
Favorite count: 0
Retweet count: 0
Text: Was sollen 150 Euro Computerzuschuss bringen? Wer wartet die Computer in den 427 NRW Kommunen (Schulträger) wer die Server vor Ort. Lernen lebt von Face to Face . Sonst könnten wir die Kinder betreut ( keine teure Lehrkraft) vor 1 Computer setzen &amp;zentral aus Düsseldorf beschulen
Favorite count: 0
Retweet count: 0


In [62]:
##Inserting the Users into MySQL 

!pip install mysql-connector-python
from mysql.connector import (connection)
import json
import mysql.connector

In [64]:
#You can download MySQL here: https://dev.mysql.com/downloads/installer/
#Connect to SQL database "localhost" after installing it.
#Note that the user and password must be defined during installation and re-used here.
#the user is "root" and the password is "gXt,:RsU#U-ws:3"
import mysql.connector

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
    print("Connected to MySQL server.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))


Connected to MySQL server.


In [79]:
# create a new database. This database is called "new_database"

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3")
cursor = connection1.cursor()
cursor.execute("CREATE DATABASE new_database")
print("Database created successfully!")


Database created successfully!


In [80]:
#create a new table. This one is called users_table

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
cursor.execute("CREATE TABLE users_table (id bigint PRIMARY KEY,name VARCHAR(255), screen_name VARCHAR(255), followers_count int)")
print("Table created successfully!")

Table created successfully!


In [81]:
#Insert the users into the Table
connection1 = mysql.connector.connect(
    host="localhost",
    user="root",
    password="gXt,:RsU#U-ws:3",
    database="new_database"
)
cursor = connection1.cursor()

used = set()

for user_id, user_data in users.items():
    sql = "INSERT INTO users_table (id, name, screen_name, followers_count) VALUES (" + str(user_id) + ",'" + user_data['name'].replace("'", "''") + "','" + user_data['screen_name'] + "'," + str(user_data['followers_count']) + ")"
    if user_id not in used:
        used.add(user_id)
    else:
        continue
    cursor.execute(sql)

connection1.commit()


In [85]:
#Present some of the users as in a dataframe
import pandas as pd

connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()

cursor.execute("USE new_database")

cursor.execute("SELECT * FROM users_table LIMIT 5")

result = cursor.fetchall()

df = pd.DataFrame(result, columns=['id', 'name', 'screen_name', 'followers_count'])

df

,id,name,screen_name,followers_count
0,767,Xeni Jardin 😷🏠,xeni,152608
1,5039,Jrome,jrome,642
2,5242,Ahmed Al Omran,ahmed,149844
3,11364,Kirti,Kits,922
4,12094,Nicholai Reinseth,Nicholai,277


In [91]:
#Prints top 10 users based on followers_count

connection1 = mysql.connector.connect(
    host="localhost",
    user="root",
    password="gXt,:RsU#U-ws:3",
    database="new_database"
)
cursor = connection1.cursor()
cursor.execute("USE new_database")

cursor.execute("SELECT * FROM users_table ORDER BY followers_count DESC LIMIT 10")

result = cursor.fetchall()

df = pd.DataFrame(result, columns=['id', 'name', 'screen_name', 'followers_count'])

df

,id,name,screen_name,followers_count
0,69183155,detikcom,detikcom,15927642
1,62513246,J.K. Rowling,jk_rowling,14608046
2,42606652,AajTak,aajtak,9704885
3,39240673,ABP News,ABPNews,9562582
4,240649814,TIMES NOW,TimesNow,9499328
5,56304605,Rajdeep Sardesai,sardesairajdeep,8947342
6,24744541,Le Monde,lemondefr,8808784
7,55507370,tvOneNews,tvOneNews,8787649
8,23343960,Kompas.com,kompascom,7678178
9,15016209,NTV,ntv,7429223


In [87]:
#print number of users (likely to be incorrect. Need to verify. Number is 80943): 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")

cursor = connection1.cursor()

# select the database
cursor.execute("USE new_database")

# execute the SQL command to count the number of rows in the table
cursor.execute("SELECT COUNT(*) FROM users_table")

# fetch the result
result = cursor.fetchone()

# print the number of users
print("Number of users in the table:", result[0])

# close the cursor and connection
cursor.close()
connection1.close()

Number of users in the table: 80943


In [88]:
#Directly from file

import json
with open('/Users/mazinrafi/Downloads/corona-out-3.json') as f:
    file_data = json.load(f)

unique_users = set()

for i in file_data:
    unique_users.add(i['user']['id'])

num_unique_users = len(unique_users)
print("Number of unique users:", num_unique_users)

Number of unique users: 80943


In [111]:
len(users)

80943

In [ ]:
#For testing purposes if needed. Do not run the cell

#Drop DB. May need to adjust the connection accordingly. 
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP DATABASE my_database"
cursor.execute(sql)


In [58]:

#Drop Table
connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="new_database")
cursor = connection1.cursor()
sql= "DROP TABLE users"
cursor.execute(sql)

#Test to see if your connection works: 

import mysql.connector
import json

try:
    connection1 = mysql.connector.connect(host="localhost", user="root", password="gXt,:RsU#U-ws:3",database="corona_out_2_db")
    print("Connected to MySQL server and database.")
except mysql.connector.Error as error:
    print("Failed to connect to MySQL server: {}".format(error))

In [109]:
#Side Notes

#need to keep the Python
#JSON Import Utility
#Primary Key needed
#Database needed and ID
# create a new database. This database is called "new_database"
